In [ ]:
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix

# Parameters
LABELS = ['no', 'yes']  # Class labels
IMG_SIZE = 250          # Image resize
np.random.seed(42)      # Numpy Seed
EPOCHS = 2              # Number of epochs
BATCH_SIZE = 32         # Batch size
KFOLD_SEED = 42         # KFold Seed
N_SPLITS = 2            # Number of KFold splits


def get_data(data_dir):
    X, y = [], []
    data_dir = Path(data_dir)
    for label in LABELS:
        path = data_dir / label
        class_num = LABELS.index(label)
        for img_file in path.iterdir():
            try:
                img_arr = cv2.imread(str(img_file))[..., ::-1]
                resized_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                X.append(resized_arr)
                y.append(class_num)
            except Exception as e:
                print(f"Error reading {img_file.name}: {e}")
    return np.array(X), np.array(y)

def augment_images(x, y):
    x_aug, y_aug = [], []

    for img, label in zip(x, y):
        x_aug.append(img)  # 1. original
        y_aug.append(label)

        # 2. random rotation
        k1 = np.random.choice([1, 2, 3])
        rotated = np.rot90(img, k1)
        x_aug.append(rotated)
        y_aug.append(label)

        # 3. different rotation (next clockwise) + flip
        k2 = 1 if k1 == 3 else k1 + 1
        rotated2 = np.rot90(img, k2)
        flipped = np.fliplr(rotated2)
        x_aug.append(flipped)
        y_aug.append(label)

    return np.array(x_aug), np.array(y_aug)



def create_model():
    model = Sequential()
    model.add(Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())

    model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())

    model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Dropout(0.4))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(model, x_train, y_train, x_val, y_val):
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

    history = model.fit(
        x_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(x_val, y_val),
        callbacks=[early_stop, lr_scheduler],
        verbose=1,
        shuffle=True
    )
    return history

def plot_metrics(history, fold):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Train Accuracy')
    plt.plot(val_acc, label='Val Accuracy')
    plt.legend()
    plt.title(f'Fold {fold} - Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Train Loss')
    plt.plot(val_loss, label='Val Loss')
    plt.legend()
    plt.title(f'Fold {fold} - Loss')
    plt.show()

def evaluate_model(model, x_val, y_val, fold):
    predictions = (model.predict(x_val) > 0.5).astype("int32").reshape(-1)
    print(classification_report(y_val.astype(int), predictions, target_names=['no', 'yes']))

    cm = confusion_matrix(y_val.astype(int), predictions)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=LABELS, yticklabels=LABELS)
    plt.title(f'Fold {fold} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

def run_kfold_pipeline():
    x_data, y_data = get_data('data')
    x_data = x_data / 255.0
    y_data = y_data.astype('float32')

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=KFOLD_SEED)

    for fold, (train_idx, val_idx) in enumerate(kf.split(x_data), 1):
        print(f"\n--- Fold {fold} ---")
        print(f"Train samples: {len(train_idx)}, Validation samples: {len(val_idx)}")
        x_train, x_val = x_data[train_idx], x_data[val_idx]
        y_train, y_val = y_data[train_idx], y_data[val_idx]

        x_train, y_train = augment_images(x_train, y_train)
        print(f"Augmented Train samples: {len(x_train)}, Validation samples: {len(x_val)}")
        model = create_model()
        print("model created.")
        # get string for the file names
        now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        checkpoint = ModelCheckpoint(f'{now}model_fold_{fold}.h5', monitor='val_accuracy', save_best_only=True)
        history = train_model(model, x_train, y_train, x_val, y_val)
        print("Training complete.")
        evaluate_model(model, x_val, y_val, fold)
        plot_metrics(history, fold)

if __name__ == "__main__":
    run_kfold_pipeline()



--- Fold 1 ---
Train samples: 300, Validation samples: 300
Augmented Train samples: 900, Validation samples: 300
model created.
Epoch 1/2
 5/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.5100 - loss: 0.7481

KeyboardInterrupt: 